In [1]:
# LCEL 문법을 사용하여 chain 을 생성
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화
model = ChatOpenAI()

# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성
prompt = PromptTemplate.from_template('{topic}에 대하여 3문장으로 설명해줘.')

# 프롬프트와 모델을 연결하여 대화 체인을 생성
chain = prompt | model | StrOutputParser()

In [2]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{'topic':'ChatGPT'},{'topic':'Instagram'}])

['ChatGPT는 인공지능 채팅 로봇으로 자연어 처리 능력을 바탕으로 다양한 주제에 대해 사용자와 대화를 나눌 수 있습니다. 사용자의 질문에 대답하거나 대화를 이어나가며 자연스러운 대화를 제공하여 정보나 엔터테인먼트를 제공합니다. ChatGPT는 지속적인 학습을 통해 점점 더 똑똑해지고 사용자들의 요구에 맞춘 맞춤형 대화를 제공합니다.',
 'Instagram은 사진과 동영상을 공유하고 타인의 게시물을 열람할 수 있는 소셜 미디어 플랫폼이다. 사용자는 팔로우를 통해 친구나 즐겨찾는 인플루언서들의 활동을 지켜볼 수 있으며, 다양한 필터와 편집 도구를 통해 자신만의 콘텐츠를 만들어 공유할 수 있다. 인스타그램 스토리, IGTV, 라이브 방송 등을 통해 다양한 형식의 콘텐츠를 소비하고 공유할 수 있다.']

In [3]:
# 다섯 개의 질문을 처리 - > 최대 3개의 작업을 동시에 처리하도록 설정
chain.batch(
    [
        {'topic':'ChatGPT'},
        {'topic':'Instagram'},
        {'topic':'Python'},
        {'topic':'Java'},
        {'topic':'Machine Learning'}
    ],
    config={'max_concurrency':3},
)

['ChatGPT는 인공지능 챗봇으로 자연어로 대화할 수 있는 기술을 제공합니다. 사용자의 질문에 대답하거나 대화를 이어나가는 등 다양한 역할을 수행할 수 있습니다. 자연스러운 대화를 위해 머신러닝 알고리즘을 사용하여 지속적으로 업데이트 되고 개선됩니다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 누구나 자신의 일상을 공유하고 팔로워들과 소통할 수 있으며 해시태그를 통해 다양한 주제의 콘텐츠를 쉽게 찾을 수 있다. 인기 있는 인플루언서들이 활동하는 플랫폼으로서 광범위한 유저들이 사용하고 있다.',
 'Python은 인터프리터 언어로 다양한 운영체제에서 사용할 수 있는 간단하고 가독성이 뛰어난 프로그래밍 언어이다. 파이썬은 높은 생산성과 뛰어난 확장성을 제공하여 데이터 분석, 머신러닝, 웹 개발 등 다양한 분야에서 널리 사용되고 있다. 코드 블록을 들여쓰기로 구분하며 간결하고 쉬운 문법을 가지고 있어 초보자부터 전문가까지 널리 사용되는 언어이다.',
 'Java는 객체지향 프로그래밍 언어로, 가장 널리 사용되는 프로그래밍 언어 중 하나이다. 자바는 플랫폼 독립적이며 다양한 운영 체제에서 실행될 수 있어 유연하고 효율적인 언어로 평가받고 있다. Java는 개발자들이 다양한 응용 소프트웨어 및 웹 애플리케이션을 개발할 때 많이 활용된다.',
 'Machine Learning은 컴퓨터 시스템이 데이터를 분석하고 패턴을 학습하여 스스로 결정을 내리는 기술이다. 이를 통해 인공지능이 과거의 데이터를 기반으로 미래의 결과를 예측하고 문제를 해결할 수 있다. 주로 지도학습, 비지도학습, 강화학습과 같은 방식으로 구현되며 다양한 분야에서 활용될 수 있다.']

In [6]:
#  Parallel 병령성
from langchain_core.runnables import RunnableParallel

# 체인 1 : {country}의 수도를 물어보는 체인
chain1 = (
    PromptTemplate.from_template('{country}의 수도는 어디야?')
    | model
    | StrOutputParser()
)

# 체인 2 : {country}의 면적을 물어보는 체인
chain2 = (
    PromptTemplate.from_template('{country}의 면적은 얼마야?')
    | model
    | StrOutputParser()
)
# 2개의 체인을 동시에 생성하는 병렬 실행 체인을 생성
combined = RunnableParallel(capital=chain1, area=chain2)




In [7]:
# 체인 1 실행
chain1.invoke({'country':'대한민국'})

'대한민국의 수도는 서울이에요.'

In [8]:
# 체인 2 실행
chain2.invoke({'country':'미국'})

'미국의 총 면적은 약 9,826,675 제곱 킬로미터입니다.'

In [9]:
# 병렬 체인을 실행
combined.invoke({'country':'대한민국'})

{'capital': '대한민국의 수도는 서울이야.', 'area': '대한민국의 면적은 약 100,210.7km² 입니다.'}

In [10]:
# 베치에서의 병렬 처리
chain1.batch([{'country':'대한민국'}, {'country':'미국'},{'country':'우즈베크스탄'}])

['대한민국의 수도는 서울이다.', '미국의 수도는 워싱턴 D.C.입니다.', '우즈베키스탄의 수도는 타슈켄트입니다.']

In [11]:
chain2.batch([{'country':'대한민국'}, {'country':'미국'},{'country':'우즈베크스탄'}])

['대한민국의 총 면적은 약 100,363㎢이다.',
 '미국의 총 면적은 약 9,826,675 km²입니다.',
 '우즈베키스탄의 면적은 약 448,978제곱킬로미터입니다.']

In [12]:
# 베치에서의 병렬 처리
combined.batch([{'country':'대한민국'}, {'country':'미국'},{'country':'우즈베크스탄'}])

[{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,284 제곱 킬로미터 입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 총 면적은 대략 9,833,517km² 입니다.'},
 {'capital': '우즈베키스탄의 수도는 탸슈켄트입니다.', 'area': '우즈베키스탄의 면적은 448,978 제곱킬로미터입니다.'}]

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template('{num}의 10는?')
llm = ChatOpenAI(temperature=0)

chain = prompt | llm

In [14]:
# chain을 실행
chain.invoke({'num': 5})

AIMessage(content='5의 10승은 9765625입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1tEkVEz6BNbOmrahBQiRcGCKr3fC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9712da08-b119-48af-8175-9cdacb0688e1-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({'num':10})


{'num': 10}

In [20]:
runnable_chain = {'num':RunnablePassthrough()} | prompt | ChatOpenAI()

runnable_chain.invoke(10)

AIMessage(content='10의 10은 10^10으로 표기되며 10을 10번 곱한 값인 10,000,000,000을 의미합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 13, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1tL9g9GBUVUA7xlFcgESLXSaj5KB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--584446f4-92c3-4054-8eba-97ece62631d6-0', usage_metadata={'input_tokens': 13, 'output_tokens': 41, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
RunnablePassthrough.assign(new_num = lambda x: x['num'] * 3).invoke({'num':1})

{'num': 1, 'new_num': 3}

In [22]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    passed = RunnablePassthrough(), # 입력된 데이터를 그데로 통과시키는 역할
    extra = RunnablePassthrough.assign(mult = lambda x: x['num'] * 3),
    # 입력된 딕셔너리의 'num'키에 해당하는 값에 1을 더한다.
    modified = lambda x:x['num'] + 1,
)

runnable.invoke({'num':1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [23]:
chain1 = (
    {'country':RunnablePassthrough()}
    | PromptTemplate.from_template('{country}의 수도는')
    | ChatOpenAI()
)
chain2 = (
    {'country':RunnablePassthrough()}
    | PromptTemplate.from_template('{country}의 면적은?')
    | ChatOpenAI() 
)

In [24]:
combined_chain = RunnableParallel(capital = chain1, area = chain2)
combined_chain.invoke('대한민국')

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 17, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1toKhXOeXlEqPRvYOqudrlkwxbQS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7305b2c9-f980-4a94-8505-618f4a1a4480-0', usage_metadata={'input_tokens': 17, 'output_tokens': 5, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,210㎢ 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45, 'c